In [ ]:
!pip install --upgrade -q langchain
!pip install google-generativeai langchain-google-genai
!pip install chromadb pypdf2 python-dotenv
!pip install PyPDF
!pip install -U langchain-community
!pip install sentence-transformers
!pip install langchainhub  

In [19]:
# Librerías Genéricas  
#from google.colab import userdata
import os
from IPython.display import Markdown

# Librerías para la preparación de datos
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings

# Librerías para el proceso de Retrieval
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI

c:\Users\flori\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
import re
import pandas as pd
from google.cloud import bigquery
import vertexai 
from vertexai.generative_models import GenerationConfig, GenerativeModel, Image, Part
import vertexai.preview.generative_models as generative_models 
import sys
import os

In [8]:
import os 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:\\Users\\flori\\OneDrive\\Desktop\\LDP\\ldp-data-genai-producto-b871ebffc681.json"

In [25]:
# Define project information
PROJECT_ID = "ldp-data-genai-producto"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}


vertexai.init(project=PROJECT_ID, location=LOCATION)

In [38]:
import requests
from langchain import LLMChain
from pydantic import BaseModel
from selenium import webdriver
#from langchain_llms import Gemini

# Configuración de Gemini (asegúrate de que esta instancia es correcta)
gemini_model = GenerativeModel("gemini-1.5-pro-002")

# Definición de clases
class Libro(BaseModel):
    """Información acerca de un libro"""
    título: str
    autor: str
    precio: str

class LibroScrapper(BaseModel):
    """Scrapper de libros"""
    url: str
    html: str

def obtener_html(url: str) -> str:
    """
    Obtener HTML crudo de una página web utilizando Selenium.
    
    Args:
        url (str): URL de la página web.
    
    Returns:
        str: HTML crudo de la página web.
    """
    driver = webdriver.Chrome()
    driver.get(url)
    html = driver.page_source
    driver.quit()
    return html

def extraer_info(html: str) -> list[Libro]:
    """
    Extraer información de libros desde el HTML crudo utilizando Gemini.
    
    Args:
        html (str): HTML crudo de la página web.
    
    Returns:
        list[Libro]: Lista de objetos Libro con la información extraída.
    """
    
    resultado = gemini_model.generate_content(
        f"""Eres un experto en hacer web scraping y analizar HTML crudo:{html}, si no se proporciona explícitamente no supongas""",
        generation_config=GenerationConfig(
            temperature=0,
            max_output_tokens=8192,
        )
    )
    
    # Accede directamente al contenido de `resultado` si es un único objeto
    texto_generado = resultado.text  # Asegúrate de que `text` es la propiedad correcta
    
    libros = []
    
    # Suponiendo que cada libro está separado por un salto de línea
    for libro in texto_generado.split("\n"):
        datos = libro.split(",")  # Suponiendo que los datos están separados por comas
        if len(datos) == 3:
            libros.append(Libro(título=datos[0], autor=datos[1], precio=datos[2]))
    
    return libros

def main():
    url = "https://www.buscalibre.pe"
    html = obtener_html(url)
    libros = extraer_info(html)
    print(libros)

if __name__ == "__main__":
    main()



[Libro(título='Claro', autor=' puedo ayudarte con eso. Como experto en web scraping y análisis de HTML sin procesar', precio=' puedo extraer los datos que necesitas de este HTML.'), Libro(título='Para extraer estos datos', autor=' puedes usar bibliotecas de web scraping en Python como Beautiful Soup y Scrapy', precio=' o herramientas similares en otros lenguajes de programación.  El proceso general sería:'), Libro(título='4. **Almacenar los datos:** Guardar los datos extraídos en un formato estructurado', autor=' como un archivo CSV', precio=' JSON o una base de datos.')]


In [46]:

gemini_model = GenerativeModel("gemini-1.5-pro-002")

# Definición de clases
class Libro(BaseModel):
    """Información acerca de un libro"""
    título: str
    autor: str
    precio: str

def obtener_html(url: str) -> str:
    """
    Obtener HTML crudo de una página web utilizando Selenium.
    
    Args:
        url (str): URL de la página web.
    
    Returns:
        str: HTML crudo de la página web.
    """
    driver = webdriver.Chrome()
    driver.get(url)
    html = driver.page_source
    driver.quit()
    return html

def extraer_info(html: str) -> list[Libro]:
    """
    Extraer información de libros desde el HTML crudo utilizando Gemini.
    
    Args:
        html (str): HTML crudo de la página web.
    
    Returns:
        list[Libro]: Lista de objetos Libro con la información extraída.
    """
    
    # Prompt mejorado
    prompt = (
        "Por favor, analiza el siguiente HTML y extrae la información de los libros. "
        "Devuélveme un JSON con una lista de libros, cada uno con los campos 'título', 'autor', y 'precio':\n"
        f"{html}"
    )
    
    resultado = gemini_model.generate_content(
        prompt,
        generation_config=GenerationConfig(
            temperature=0,
            max_output_tokens=8192,
        )
    )
    
    # Supongamos que la respuesta es un JSON string
    libros_data = resultado.text.strip()  # Limpiamos espacios en blanco
    libros = []
    
    # Parsear el JSON
    try:
        libros_json = json.loads(libros_data)
        for libro in libros_json:
            libros.append(Libro(**libro))
    except json.JSONDecodeError:
        print("Error al decodificar el JSON:", libros_data)
    
    return libros

def main():
    url = "https://www.buscalibre.pe"
    html = obtener_html(url)
    libros = extraer_info(html)
    print(libros)

if __name__ == "__main__":
    main()


Error al decodificar el JSON: ```json
[
  {
    "título": "Alas de Ónix - Saga Empíreo",
    "autor": "Rebecca Yarros",
    "precio": "109.90"
  },
  {
    "título": "Behave: The Biology of Humans at our Best and Worst (en Inglés)",
    "autor": "Robert M Sapolsky",
    "precio": "78.57"
  },
  {
    "título": "Globalismo: Ingeniería Social y Control Total en el Siglo xxi",
    "autor": "Agustin Laje",
    "precio": "84.98"
  },
  {
    "título": "El Buzón de las Impuras",
    "autor": "Francisca Solar García",
    "precio": "99.60"
  },
  {
    "título": "Berserk Deluxe Volume 4 (en Inglés)",
    "autor": "Miura, Kentaro ; Miura, Kentaro ; Johnson, Duane",
    "precio": "160.72"
  },
  {
    "título": "Mitos y leyendas de América",
    "autor": "Carlos Garayar y Jéssica Rodríguez",
    "precio": "50.40"
  },
  {
    "título": "Estuche Saga Dune 1-6. La Mayor Epopeya de Todos Los Tiempos / Dune Saga Books 1-6. the Greatest Epic Adventure of All Time (Boxed Collection)",
    "autor": "H

In [ ]:

gemini_model = GenerativeModel("gemini-1.5-pro-002")

# Definición de clases
class Libro(BaseModel):
    """Información acerca de un libro"""
    título: str
    autor: str
    precio: str

def obtener_html(url: str) -> str:
    """
    Obtener HTML crudo de una página web utilizando Selenium.
    
    Args:
        url (str): URL de la página web.
    
    Returns:
        str: HTML crudo de la página web.
    """
    driver = webdriver.Chrome()
    driver.get(url)
    html = driver.page_source
    driver.quit()
    return html

def extraer_info(html: str) -> list[Libro]:
    """
    Extraer información de libros desde el HTML crudo utilizando Gemini.
    
    Args:
        html (str): HTML crudo de la página web.
    
    Returns:
        list[Libro]: Lista de objetos Libro con la información extraída.
    """
    
    # Prompt mejorado
    prompt = (
        "Por favor, analiza el siguiente HTML y extrae la información de los libros. "
        "Devuélveme un JSON con una lista de libros, cada uno con los campos 'título', 'autor', y 'precio':\n"
        f"{html}"
    )
    
    resultado = gemini_model.generate_content(
        prompt,
        generation_config=GenerationConfig(
            temperature=0,
            max_output_tokens=8192,
        )
    )
    
    # Supongamos que la respuesta es un JSON string
    libros_data = resultado.text.strip()  # Limpiamos espacios en blanco
    libros = []
    
    # Parsear el JSON
    try:
        libros_json = json.loads(libros_data)
        for libro in libros_json:
            libros.append(Libro(**libro))
    except json.JSONDecodeError:
        print("Error al decodificar el JSON:", libros_data)
    
    return libros

def main():
    url = "https://www.buscalibre.pe"
    html = obtener_html(url)
    libros = extraer_info(html)
    print(libros)

if __name__ == "__main__":
    main()


In [ ]:
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

def guardar_cookies(driver, filename):
    cookies = driver.get_cookies()
    with open(filename, 'w') as file:
        json.dump(cookies, file)

def cargar_cookies(driver, filename):
    with open(filename, 'r') as file:
        cookies = json.load(file)
        for cookie in cookies:
            driver.add_cookie(cookie)

# Configurar Selenium
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ejecutar en segundo plano
driver = webdriver.Chrome(service=Service(), options=chrome_options)

# Iniciar sesión manualmente y guardar cookies
driver.get("https://www.linkedin.com/home")
input("Inicie sesión en LinkedIn y luego presione Enter...")
guardar_cookies(driver, 'cookies.json')
driver.quit()


In [44]:


# Definición de clases
class Libro(BaseModel):
    """Información acerca de un libro"""
    título: str
    autor: str
    precio: str

def obtener_html(url: str) -> str:
    """
    Obtener HTML crudo de una página web utilizando Selenium.
    
    Args:
        url (str): URL de la página web.
    
    Returns:
        str: HTML crudo de la página web.
    """
    driver = webdriver.Chrome()
    driver.get(url)
    html = driver.page_source
    driver.quit()
    return html

def extraer_info(html: str) -> list[Libro]:
    """
    Extraer información de libros desde el HTML crudo utilizando Gemini.
    
    Args:
        html (str): HTML crudo de la página web.
    
    Returns:
        list[Libro]: Lista de objetos Libro con la información extraída.
    """
    
    # Prompt mejorado
    prompt = (
        "Por favor, analiza el siguiente HTML y extrae la información de los libros. "
        "Devuélveme un JSON con una lista de libros, cada uno con los campos 'título', 'autor', y 'precio':\n"
        f"{html}"
    )
    
    resultado = gemini_model.generate_content(
        prompt,
        generation_config=GenerationConfig(
            temperature=0,
            max_output_tokens=8192,
        )
    )
    
    # Supongamos que la respuesta es un JSON string
    libros_data = resultado.text.strip()  # Limpiamos espacios en blanco
    libros = []
    
    # Intentar cargar el JSON
    try:
        libros_json = json.loads(libros_data)
        libros = [Libro(**libro) for libro in libros_json]  # Crear lista de libros
    except json.JSONDecodeError as e:
        print("Error al decodificar el JSON:", e)
        print("Respuesta del modelo:", libros_data)
    
    return libros

def main():
    url = "https://www.buscalibre.pe"
    html = obtener_html(url)
    libros = extraer_info(html)
    print(libros)

if __name__ == "__main__":
    main()


Error al decodificar el JSON: Expecting value: line 1 column 1 (char 0)
Respuesta del modelo: ```json
[]
```

El HTML proporcionado no contiene información de libros en un formato que pueda ser fácilmente extraído para crear el JSON deseado.  El HTML muestra la página principal de Buscalibre Perú, con banners, menús de navegación y algunas sugerencias de libros, pero los datos de 'título', 'autor' y 'precio' están embebidos dentro de la estructura HTML de una manera que requiere un análisis más complejo (parsing) con una herramienta específica o una librería de web scraping.

Para extraer la información de los libros, necesitarías usar una librería como Beautiful Soup (en Python) o Cheerio (en Node.js) para analizar el HTML y extraer los datos de los elementos relevantes (etiquetas `<h3>` para el título, `<div class="autor">` para el autor y `<p class="precio-ahora">` para el precio).

**Ejemplo con Beautiful Soup (Python):**

```python
import requests
from bs4 import BeautifulSoup
imp